In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path.append('/Users/lex/miniconda2/lib/python2.7/site-packages')
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,TimeDistributed,Flatten,Masking
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import util.util as util

In [3]:
df_rf = pd.read_csv('rf_prediction.csv')
df_market = pd.read_csv('sp_prediction.csv')
df_fb = pd.read_csv('fb_prediction.csv')

In [6]:
df_rf.columns = ['date', 'open', 'high', 'low', 'close']
df_rf.head()

,date,open,high,low,close
0,2013-03-15,2.035,2.045,1.991,1.991
1,2013-03-18,1.908,1.965,1.898,1.955
2,2013-03-19,1.951,1.970,1.892,1.904
3,2013-03-20,1.904,1.962,1.899,1.958
4,2013-03-21,1.956,1.967,1.910,1.915


In [9]:
df_rf.date = pd.to_datetime(df_rf.date)

In [14]:
risk_free_rate = np.average(df_rf[-90:max(df_rf.index)].open)

In [15]:
risk_free_rate

2.0100570396529212

In [18]:
df_market.head()

,Unnamed: 0,open,high,low,close
0,2013-04-17,1574.569946,1574.569946,1543.689941,1552.010010
1,2013-04-18,1552.030029,1554.380005,1536.030029,1541.609985
2,2013-04-19,1541.609985,1555.890015,1539.400024,1555.250000
3,2013-04-22,1555.250000,1565.550049,1548.189941,1562.500000
4,2013-04-23,1562.500000,1579.579956,1562.500000,1578.780029


In [34]:
df_market['avg_return'] = 0.0
avg_return_list = []
for i, row in df_market.iterrows():
    if i > 365:
        last_year = df_market.open.iloc[i - 365]
        year_return = (row.open - last_year)/last_year
        avg_return_list.append(year_return)
    else:
        avg_return_list.append(0.0)

In [35]:
df_market.avg_return = avg_return_list

In [42]:
market_rate = np.average(avg_return_list[-90:max(df_market.index)])

In [43]:
df_fb.head()

,Unnamed: 0,open,high,low,close
0,2013-04-17,26.64,27.20,26.39,26.625
1,2013-04-18,26.82,26.82,25.15,25.690
2,2013-04-19,25.56,25.96,25.33,25.730
3,2013-04-22,25.81,26.36,25.70,25.970
4,2013-04-23,26.20,26.33,25.77,25.980


In [47]:
fb_return_list = []
for i, row in df_fb.iterrows():
    if i > 365:
        last_year = df_fb.open.iloc[i - 365]
        year_return = (row.open - last_year)/last_year
        fb_return_list.append(year_return)
    else:
        fb_return_list.append(0.0)

In [48]:
fb_return_list

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [49]:
fb_rate = np.average(fb_return_list[-90:max(df_fb.index)])
fb_rate

0.34240752342901021

In [59]:
np.std(fb_return_list[-90:max(df_fb.index)])

0.065254294938916033

In [55]:
risk_free_rate = risk_free_rate*0.01

In [56]:
top = fb_rate - risk_free_rate
bottom = market_rate - risk_free_rate

In [57]:
beta = top/bottom
beta

1.7831531465469515

In [58]:
risk_free_rate + beta*(bottom)

0.34240752342901026